In [1]:
import gdal
import osr
import csv
import os.path as path
import pandas as pd
import netCDF4
import datetime
import xarray as xr
import gdal
import subprocess
import glob
from Modelo.Funcoes.RasterTools.RasterToCSVeVRT import RasterToCSVeVRT
from Modelo.beans import SerialFile, TableData, FileData
from Modelo.beans.RasterData import RasterFile
from lxml import etree
from datetime import datetime
from datetime import timedelta 
import numpy as np

<h1> Passar arquivos nc pra diário e separar as variáveis </h1>

In [3]:
# Mostra as variáveis presentes, precisa saber o nome da variável

nc_file = path.join(r"D:\\1-Dados Climaticos\\ecmwf\\2008-2019_e_D.nc") #  ← nome do arquivo

nc = netCDF4.Dataset(nc_file, 'r') # abre o arquivo
print(nc.variables.keys()) # printa os nomes das variéveis presentes no arquivo

FileNotFoundError: [Errno 2] No such file or directory: b'D:\\\\1-Dados Climaticos\\\\ecmwf\\\\2008-2019_e_D.nc'

In [90]:
# Passar arquivos nc pra diário e separar as variáveis

variavel = 'e' #  ← escolha a variavel da lista anterior
ressample = '1D' # ← esolhe a frequencia do ressample, no caso pra 1 dia

ano = '2009'
out_nc_file = path.join(r"C:\Data\ecmwf\TotalPrecipitation\tp_9km_stack", ano + '_tp_daily.nc')

with xr.open_dataset(nc_file) as nc:
    nc_resampled = nc.sel(time=ano)[[variavel]].resample(time='1D', keep_attrs=True).mean()
    
nc_resampled.to_netcdf(out_nc_file)

<h1> Separando Stack para camadas individuais </h1>

In [2]:
# Script para separar os rasters multbant para single band
ano = '2019'

tif_stack = FileData(file_full_path = 'C:\\Data\\ecmwf\\3-SKT\\2019_t_daily.tif')

daily_9km_folder = 'C:\\Data\\ecmwf\\3-SKT\\2-SKT_9km_daily\\'

    
n_bands = gdal.Open(tif_stack.file_full_path).RasterCount
    
print("Numero de bandas:", n_bands)
    
child_processes = []

import io


ano_init = 2019

with io.open("log_out_Prec_daily.txt", mode='wb') as out:
        
    for b in range(0, n_bands, 36):
        
        for ii in range(0, 36):
            
            i = b+ii

            file_data_time = datetime.strptime(str(ano_init), "%Y") + timedelta(days=i)

            formated_file_data_time = file_data_time.strftime("%Y-%m-%d")

            out_file = FileData(file_path = daily_9km_folder, 
                                file_name = (formated_file_data_time), file_ext = ".tif")
            out_file.file_ext = ".tif"

            s = "gdal_translate " + tif_stack.file_full_path + " -b " + str(i+1) + " " + out_file.file_full_path

            p = subprocess.Popen(s, stdout=out, stderr=out)
            child_processes.append(p)    # start this one, and immediately return to start another

            #subprocess.call(s, shell=True)
            print(s)
        
        # now you can join them together
        for cp in child_processes:
            cp.wait()                         # this will block on each child process until it exits


    with io.open("log_out.txt", mode='wb') as out:
    
        for b in range(b, n_bands):
        
            i = b

            file_data_time = datetime.strptime(str(ano_init), "%Y") + timedelta(days=i)

            formated_file_data_time = file_data_time.strftime("%Y-%m-%d")

            out_file = FileData(file_path = daily_9km_folder, 
                                file_name = (formated_file_data_time), file_ext = ".tif")
            out_file.file_ext = ".tif"

            s = "gdal_translate " + tif_stack.file_full_path + " -b " + str(i+1) + " " + out_file.file_full_path

            p = subprocess.Popen(s, stdout=out, stderr=out)
            child_processes.append(p)    # start this one, and immediately return to start another

            #subprocess.call(s, shell=True)
            print(s)
        
    # now you can join them together
    for cp in child_processes:
        cp.wait()                         # this will block on each child process until it exits

Numero de bandas: 365
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 1 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-01-01.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 2 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-01-02.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 3 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-01-03.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 4 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-01-04.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 5 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-01-05.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 6 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-01-06.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 7 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-01-07.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 8 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-01-08.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 9 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-01-09.tif
gdal_t

gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 109 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-04-19.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 110 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-04-20.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 111 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-04-21.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 112 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-04-22.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 113 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-04-23.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 114 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-04-24.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 115 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-04-25.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 116 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-04-26.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 117 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-04-27.tif
gdal_trans

gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 195 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-07-14.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 196 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-07-15.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 197 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-07-16.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 198 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-07-17.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 199 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-07-18.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 200 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-07-19.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 201 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-07-20.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 202 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-07-21.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 203 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-07-22.tif
gdal_trans

gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 272 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-09-29.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 273 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-09-30.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 274 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-10-01.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 275 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-10-02.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 276 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-10-03.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 277 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-10-04.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 278 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-10-05.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 279 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-10-06.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 280 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-10-07.tif
gdal_trans

gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 361 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-12-27.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 362 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-12-28.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 363 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-12-29.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 364 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-12-30.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 365 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-12-31.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 366 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2020-01-01.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 367 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2020-01-02.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 368 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2020-01-03.tif
gdal_translate C:\Data\ecmwf\3-SKT\2019_t_daily.tif -b 369 C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2020-01-04.tif
gdal_trans

datetime.datetime(2008, 1, 2, 0, 0)

<h1> Interpolar para 1km </h1>

In [4]:
# Script para criar os arquivos CSV e VRT
# Folder parameters

#daily_9km_folder = "C:\\Data\\ecmwf\\3-SKT\\2-SKT_9km_daily\\"
daily_vrt_csv_folder = "C:\\Data\ecmwf\\3-SKT\\3-SKT_CSV_VRT"


img_teste = SerialFile(root_path=daily_9km_folder, root_filter=".tif")
img_teste.loadListByRoot()

print(img_teste[0].file_full_path)
print(img_teste[-1].file_full_path)

paramIn = dict()
paramIn["images"] = img_teste
paramIn["out_folder"] = daily_vrt_csv_folder

print(len(img_teste))

f = RasterToCSVeVRT()
f.__LoadParams__(paramIn)

creat_vrt_csv(f) # gera os arquivos

endereco das imagens:  C:\Data\ecmwf\3-SKT\2-SKT_9km_daily
C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-01-01.tif
C:\Data\ecmwf\3-SKT\2-SKT_9km_daily\2019-12-31.tif
365
endereco das imagens:  C:\Data\ecmwf\3-SKT\2-SKT_9km_daily
2019-01-01
2019-01-02
2019-01-03
2019-01-04
2019-01-05
2019-01-06
2019-01-07
2019-01-08
2019-01-09
2019-01-10
2019-01-11
2019-01-12
2019-01-13
2019-01-14
2019-01-15
2019-01-16
2019-01-17
2019-01-18
2019-01-19
2019-01-20
2019-01-21
2019-01-22
2019-01-23
2019-01-24
2019-01-25
2019-01-26
2019-01-27
2019-01-28
2019-01-29
2019-01-30
2019-01-31
2019-02-01
2019-02-02
2019-02-03
2019-02-04
2019-02-05
2019-02-06
2019-02-07
2019-02-08
2019-02-09
2019-02-10
2019-02-11
2019-02-12
2019-02-13
2019-02-14
2019-02-15
2019-02-16
2019-02-17
2019-02-18
2019-02-19
2019-02-20
2019-02-21
2019-02-22
2019-02-23
2019-02-24
2019-02-25
2019-02-26
2019-02-27
2019-02-28
2019-03-01
2019-03-02
2019-03-03
2019-03-04
2019-03-05
2019-03-06
2019-03-07
2019-03-08
2019-03-09
2019-03-10
2019-03-11
2019-0

2019-12-29
2019-12-30
2019-12-31


{'CSVs': [<Modelo.beans.FileData.FileData at 0x20c9e1990d0>,
 'VRTs': [<Modelo.beans.FileData.FileData at 0x20c9e1990d0>,
  <Modelo.beans.FileData.FileData at 0x20c9e2b7dc0>]}

In [3]:
  def creat_vrt_csv(self):
        
        imagensIN = self.paramentrosIN_carregados["images"]
        outFolder = self.paramentrosIN_carregados["out_folder"]
        
        imagensIN.loadListByRoot()
        
        listaCSV = SerialFile()
        listaVRT = SerialFile()
        
        n_imagens = len(imagensIN)
        i_imagem = 0
        self.setProgresso(i_imagem, n_imagens)
        
        for img in imagensIN :

            matriz = img.loadRasterData()
            nome_img = img.file_name
            info = img.getRasterInformation()
    
            xmin = float(info["xmin"])
            xmax = float(info["xmax"])
            nx = float(info["nx"])
            x_pixelSize = (xmax - xmin) / nx
        
            ymin = float(info["ymin"])
            ymax = float(info["ymax"])
            ny = float(info["ny"])
            y_pixelSize = (ymax - ymin) / ny    
        
            n_linhas = len(matriz)
            n_colunas = len(matriz[0]) 
            
            csv_file = FileData(file_path=outFolder, file_name = nome_img, ext = "csv")
            file_csv_path = csv_file.file_full_path
            
            init_y_position = ymin + (y_pixelSize/2)
            init_x_position = xmin + (x_pixelSize/2)
            
            with open(file_csv_path,'w') as csv:
                csv.write("Easting,Northing,Value\n")
                for i_linha in range(0, n_linhas):
                        
                    cy = init_y_position + (y_pixelSize * i_linha)
                    cy = str(cy)
                    for i_coluna in range(0, n_colunas):
                        
                        value = matriz[i_linha][i_coluna]

                        cx = init_x_position + (x_pixelSize * i_coluna)
                        line = str(cx) + ',' + cy + ',' + str(value) + '\n'
                        csv.write(line)

            listaCSV.append(csv_file)
            
            vrt_file = csv_file
            vrt_file.file_ext = "vrt"
            
            print(vrt_file.file_name)
                
            root = etree.Element("OGRVRTDataSource")
            csv_node = etree.Element("OGRVRTLayer", name=csv_file.file_name)   
            root.append(csv_node)
                                      
            csv_node.append(etree.XML("<SrcDataSource>" +file_csv_path+"</SrcDataSource>"))
            csv_node.append(etree.XML("<GeometryType>wkbPoint</GeometryType>"))
            csv_node.append(etree.XML('<GeometryField encoding="PointFromColumns" x="Easting" y="Northing" z="Value"/>'))
                
            tree = etree.ElementTree(root)
            tree.write(vrt_file.file_full_path, pretty_print=True)
            
            listaVRT.append(vrt_file)
            
            i_imagem = i_imagem + 1.0

        saida = TableData()
        saida["CSVs"] = listaCSV
        saida["VRTs"] = listaVRT
        
        return saida

In [9]:
#Interpolation parameters:

parameters_file = "C:\\Data\\ecmwf\\3-SKT\\parametros_cv_skt.csv"
parameters = pd.read_csv(parameters_file, sep=';')

data_id = "data"
power_id = "idp"
maxdist_id = "maxdist"
variable_id = "var"
variable = "skt"

parameters = parameters[parameters[variable_id] == variable]
parameters_dict = parameters.to_dict(orient="records")[4018:]

print("numero de registros:", len(parameters_dict), "primeira linha:")
print(parameters_dict[0])


##########################################################
# ou pode ser fixado

# parameters = {'power_id': '2,5',        # força do IDW
#               'maxdist_id': 50000.0,  # distância em metros
#              }

numero de registros: 366 primeira linha:
{'idp': '2,5', 'maxdist': 300000.0, 'MAE': '5,93477011271531e-05', 'MRSE': '2,67725137964214', 'corObs': '0,79890663496315', 'corRes': '-0,279844707752424', 'ind': '0,00015888871472126', 'data': '2019-01-01', 'var': 'skt'}


In [2]:
# Conf imagem

#n_linhas, n_colunas = "183", "117" ## para 4km
#n_linhas, n_colunas = "367", "234" ## para 2km
#n_linhas, n_colunas = "734", "468" ## para 1km
n_linhas, n_colunas = "2938", "1872" ## para 250m
limites_x_min, limites_x_max = "-54.6182781" , "-48.0224681"
limites_y_min, limites_y_max = "-22.5165800", "-26.7192200"
projection = 'EPSG:4326'

# ------------- Dados da imagem original ------------------------------

original_n_linhas, original_n_colunas = 66, 43
limites_x_min_o, limites_x_max_o = "-54.6600006099999973" , "-48.0600006100000030"
limites_y_min_o, limites_y_max_o = "-26.7599990709999993" , "-22.4600002429999996"
pixel_res_meters = 9000.0 

comprimento_degree = (float(limites_x_max_o) - float(limites_x_min_o))
altura_degree = (float(limites_y_max_o) - float(limites_y_min_o))

comprimento_pixel_original_degree = comprimento_degree / 66
altura_pixel_original_degree = altura_degree / 43

print (comprimento_pixel_original_degree)
print (altura_pixel_original_degree)

# function to convert meters in degrees based on image settings
def meters_to_degrees(dist_meters):
    mean_larg_ = (comprimento_pixel_original_degree + altura_pixel_original_degree) / 2.0
    return ((dist_meters * mean_larg_) / pixel_res_meters) # retur dist in degrees

meters_to_degrees(50000)


0.09999999999999991
0.09999997274418604


0.5555554798449609

In [19]:
# conf algoritimo
def str_algoritimo_configuration(row):
    
    radius = meters_to_degrees(float(row[maxdist_id]))
    if (radius == np.Inf): radius = 0.5
    
    str_algoritimo_conf = ":power=" + str(row[power_id])
#     str_algoritimo_conf = ":power=2.5"
    str_algoritimo_conf += ":radius=" + str(radius)
#     str_algoritimo_conf += ":radius=0.55" 
#     str_algoritimo_conf += ":max_points=" + str(row[maxdist_id])
#     str_algoritimo_conf += ":nodata=-9999"
    return str_algoritimo_conf

In [12]:
def build_exec_string(vrt_file, csv_layer, out_file, conf_row):
    string_execucao = ['gdal_grid',
                       '-a', 'invdistnn' + str_algoritimo_configuration(conf_row),
                       '-txe', limites_x_min, limites_x_max,
                       '-tye', limites_y_min, limites_y_max,
                       '-outsize', n_linhas, n_colunas, 
                       '-of', 'GTiff',
                       '-ot', 'Float32',
                       '-a_srs', projection,
                       '-l', csv_layer, vrt_file, out_file]
    
    return string_execucao

In [22]:
# só pra checar a aquantidade de arquivos
#daily_vrt_csv_folder = "C:\\Data\\ecmwf\\3-SKT\\3-SKT_CSV_VRT\\"
vrt_list = glob.glob(path.join(daily_vrt_csv_folder,'*.vrt'))

print("Numero de arquivos VRT:", len(vrt_list))
print("Numero de linhas no arquivo de parametros:", len(parameters_dict))

print(vrt_list[0])
print(vrt_list[-1])

print(parameters_dict[0])
print(parameters_dict[-1])

#print(parameters_dict[0])

Numero de arquivos VRT: 72
Numero de linhas no arquivo de parametros: 366
C:\Data\ecmwf\3-SKT\3-SKT_CSV_VRT\2019-01-01.vrt
C:\Data\ecmwf\3-SKT\3-SKT_CSV_VRT\2019-03-13.vrt
{'idp': '2,5', 'maxdist': 300000.0, 'MAE': '5,93477011271531e-05', 'MRSE': '2,67725137964214', 'corObs': '0,79890663496315', 'corRes': '-0,279844707752424', 'ind': '0,00015888871472126', 'data': '2019-01-01', 'var': 'skt'}
{'idp': '2,5', 'maxdist': 50000.0, 'MAE': '0,000257527467222766', 'MRSE': '0,182770077770515', 'corObs': '0,98823580668938', 'corRes': '0,15423308302869', 'ind': '4,70683152123488e-05', 'data': '2020-01-01', 'var': 'skt'}


In [23]:
# Script para fazer a interpolação
import subprocess

out_path = "C:\\Data\\ecmwf\\3-SKT\\4-SKT_1km"

ii = 0

for vrt in vrt_list:
    
    
    vrt_file = FileData(file_full_path = vrt)
    
    out = out_path + "\\" + vrt_file.file_name + ".tif"
    
    print(out)
    print(ii)
    
#     row = parameters # caso seja parametro fixo
    row = parameters_dict [ii] # caso seja parametro variável
    
    exec_string = build_exec_string(vrt_file.file_full_path, vrt_file.file_name,  out, row)
    print(exec_string)
    subprocess.call(exec_string, shell=True)
    
    ii += 1
    
    

C:\Data\ecmwf\3-SKT\4-SKT_1km\2019-01-01.tif
0
['gdal_grid', '-a', 'invdistnn:power=2,5:radius=3.3333328790697663', '-txe', '-54.6182781', '-48.0224681', '-tye', '-22.5165800', '-26.7192200', '-outsize', '734', '468', '-of', 'GTiff', '-ot', 'Float32', '-a_srs', 'EPSG:4326', '-l', '2019-01-01', 'C:\\Data\\ecmwf\\3-SKT\\3-SKT_CSV_VRT\\2019-01-01.vrt', 'C:\\Data\\ecmwf\\3-SKT\\4-SKT_1km\\2019-01-01.tif']
C:\Data\ecmwf\3-SKT\4-SKT_1km\2019-01-02.tif
1
['gdal_grid', '-a', 'invdistnn:power=3:radius=0.5555554798449609', '-txe', '-54.6182781', '-48.0224681', '-tye', '-22.5165800', '-26.7192200', '-outsize', '734', '468', '-of', 'GTiff', '-ot', 'Float32', '-a_srs', 'EPSG:4326', '-l', '2019-01-02', 'C:\\Data\\ecmwf\\3-SKT\\3-SKT_CSV_VRT\\2019-01-02.vrt', 'C:\\Data\\ecmwf\\3-SKT\\4-SKT_1km\\2019-01-02.tif']
C:\Data\ecmwf\3-SKT\4-SKT_1km\2019-01-03.tif
2
['gdal_grid', '-a', 'invdistnn:power=3:radius=0.5555554798449609', '-txe', '-54.6182781', '-48.0224681', '-tye', '-22.5165800', '-26.7192200', '-

C:\Data\ecmwf\3-SKT\4-SKT_1km\2019-01-22.tif
21
['gdal_grid', '-a', 'invdistnn:power=3,5:radius=0.8888887677519376', '-txe', '-54.6182781', '-48.0224681', '-tye', '-22.5165800', '-26.7192200', '-outsize', '734', '468', '-of', 'GTiff', '-ot', 'Float32', '-a_srs', 'EPSG:4326', '-l', '2019-01-22', 'C:\\Data\\ecmwf\\3-SKT\\3-SKT_CSV_VRT\\2019-01-22.vrt', 'C:\\Data\\ecmwf\\3-SKT\\4-SKT_1km\\2019-01-22.tif']
C:\Data\ecmwf\3-SKT\4-SKT_1km\2019-01-23.tif
22
['gdal_grid', '-a', 'invdistnn:power=3,5:radius=0.8888887677519376', '-txe', '-54.6182781', '-48.0224681', '-tye', '-22.5165800', '-26.7192200', '-outsize', '734', '468', '-of', 'GTiff', '-ot', 'Float32', '-a_srs', 'EPSG:4326', '-l', '2019-01-23', 'C:\\Data\\ecmwf\\3-SKT\\3-SKT_CSV_VRT\\2019-01-23.vrt', 'C:\\Data\\ecmwf\\3-SKT\\4-SKT_1km\\2019-01-23.tif']
C:\Data\ecmwf\3-SKT\4-SKT_1km\2019-01-24.tif
23
['gdal_grid', '-a', 'invdistnn:power=3:radius=0.5555554798449609', '-txe', '-54.6182781', '-48.0224681', '-tye', '-22.5165800', '-26.7192200

C:\Data\ecmwf\3-SKT\4-SKT_1km\2019-02-12.tif
42
['gdal_grid', '-a', 'invdistnn:power=3,5:radius=0.8888887677519376', '-txe', '-54.6182781', '-48.0224681', '-tye', '-22.5165800', '-26.7192200', '-outsize', '734', '468', '-of', 'GTiff', '-ot', 'Float32', '-a_srs', 'EPSG:4326', '-l', '2019-02-12', 'C:\\Data\\ecmwf\\3-SKT\\3-SKT_CSV_VRT\\2019-02-12.vrt', 'C:\\Data\\ecmwf\\3-SKT\\4-SKT_1km\\2019-02-12.tif']
C:\Data\ecmwf\3-SKT\4-SKT_1km\2019-02-13.tif
43
['gdal_grid', '-a', 'invdistnn:power=3,5:radius=0.8888887677519376', '-txe', '-54.6182781', '-48.0224681', '-tye', '-22.5165800', '-26.7192200', '-outsize', '734', '468', '-of', 'GTiff', '-ot', 'Float32', '-a_srs', 'EPSG:4326', '-l', '2019-02-13', 'C:\\Data\\ecmwf\\3-SKT\\3-SKT_CSV_VRT\\2019-02-13.vrt', 'C:\\Data\\ecmwf\\3-SKT\\4-SKT_1km\\2019-02-13.tif']
C:\Data\ecmwf\3-SKT\4-SKT_1km\2019-02-14.tif
44
['gdal_grid', '-a', 'invdistnn:power=3,5:radius=0.5555554798449609', '-txe', '-54.6182781', '-48.0224681', '-tye', '-22.5165800', '-26.71922

C:\Data\ecmwf\3-SKT\4-SKT_1km\2019-03-05.tif
63
['gdal_grid', '-a', 'invdistnn:power=3,5:radius=0.5555554798449609', '-txe', '-54.6182781', '-48.0224681', '-tye', '-22.5165800', '-26.7192200', '-outsize', '734', '468', '-of', 'GTiff', '-ot', 'Float32', '-a_srs', 'EPSG:4326', '-l', '2019-03-05', 'C:\\Data\\ecmwf\\3-SKT\\3-SKT_CSV_VRT\\2019-03-05.vrt', 'C:\\Data\\ecmwf\\3-SKT\\4-SKT_1km\\2019-03-05.tif']
C:\Data\ecmwf\3-SKT\4-SKT_1km\2019-03-06.tif
64
['gdal_grid', '-a', 'invdistnn:power=3,5:radius=0.5555554798449609', '-txe', '-54.6182781', '-48.0224681', '-tye', '-22.5165800', '-26.7192200', '-outsize', '734', '468', '-of', 'GTiff', '-ot', 'Float32', '-a_srs', 'EPSG:4326', '-l', '2019-03-06', 'C:\\Data\\ecmwf\\3-SKT\\3-SKT_CSV_VRT\\2019-03-06.vrt', 'C:\\Data\\ecmwf\\3-SKT\\4-SKT_1km\\2019-03-06.tif']
C:\Data\ecmwf\3-SKT\4-SKT_1km\2019-03-07.tif
65
['gdal_grid', '-a', 'invdistnn:power=3,5:radius=0.8888887677519376', '-txe', '-54.6182781', '-48.0224681', '-tye', '-22.5165800', '-26.71922

In [4]:
def build_exec_string_warp(in_file, out_file):
    string_execucao = ['gdalwarp',
                       in_file, out_file,
                       '-te', limites_x_min, limites_y_max, limites_x_max, limites_y_min,
                       '-r', 'near',
                       '-ts', n_linhas, n_colunas,
                       '-of', 'GTiff',
                       '-ot', 'Float32',
                       '-s_srs', 'EPSG:4326',
                       '-t_srs', 'EPSG:4326', 
                      ]
    
    return string_execucao

In [5]:
import glob
from os import path
rasters_list = glob.glob(path.join(r"C:\Users\Administrator\Desktop\4-SKT_1km",'*.tif'))
print(len(rasters_list))

129


In [6]:
#%%time
import io
# Script para fazer o warp de 1km pra 250m
import subprocess
from Modelo.beans import FileData

out_path = r"C:\\Data\\1-FAO ESTIMATION\\temperature"

n_files = len(rasters_list)
    
child_processes = []
with io.open("log_out_Prec_warp.txt", mode='wb') as out:
        
    for b in range(0, n_files, 36):
        
        for i in range(0, 36):
            
            ii = b+i
    
            in_file = FileData(file_full_path = rasters_list[ii])

            out = out_path + "\\" + in_file.file_name + ".tif"

            print(out)
            print(ii)

            s = build_exec_string_warp(in_file.file_full_path, out)

            print(s)

            p = subprocess.Popen(s)
            child_processes.append(p)    # start this one, and immediately return to start another

            #subprocess.call(s, shell=True)
            #subprocess.call(exec_string, shell=True)

            ii += 1

        # now you can join them together
        for cp in child_processes:
            cp.wait()                         # this will block on each child process until it exits
            
    for ii in range(ii, n_files):
    
            in_file = FileData(file_full_path = rasters_list[ii])

            out = out_path + "\\" + in_file.file_name + ".tif"

            print(out)
            print(ii)

            s = build_exec_string_warp(in_file.file_full_path, out)

            print(s)

            p = subprocess.Popen(s)
            child_processes.append(p)    # start this one, and immediately return to start another
            #subprocess.call(exec_string, shell=True)

            ii += 1
            
# now you can join them together
for cp in child_processes:
    cp.wait()                         # this will block on each child process until it exits
            


C:\\Data\\1-FAO ESTIMATION\\temperature\2017-08-25.tif
0
['gdalwarp', 'C:\\Users\\Administrator\\Desktop\\4-SKT_1km\\2017-08-25.tif', 'C:\\\\Data\\\\1-FAO ESTIMATION\\\\temperature\\2017-08-25.tif', '-te', '-54.6182781', '-26.7192200', '-48.0224681', '-22.5165800', '-r', 'near', '-ts', '2938', '1872', '-of', 'GTiff', '-ot', 'Float32', '-s_srs', 'EPSG:4326', '-t_srs', 'EPSG:4326']
C:\\Data\\1-FAO ESTIMATION\\temperature\2017-08-26.tif
1
['gdalwarp', 'C:\\Users\\Administrator\\Desktop\\4-SKT_1km\\2017-08-26.tif', 'C:\\\\Data\\\\1-FAO ESTIMATION\\\\temperature\\2017-08-26.tif', '-te', '-54.6182781', '-26.7192200', '-48.0224681', '-22.5165800', '-r', 'near', '-ts', '2938', '1872', '-of', 'GTiff', '-ot', 'Float32', '-s_srs', 'EPSG:4326', '-t_srs', 'EPSG:4326']
C:\\Data\\1-FAO ESTIMATION\\temperature\2017-08-27.tif
2
['gdalwarp', 'C:\\Users\\Administrator\\Desktop\\4-SKT_1km\\2017-08-27.tif', 'C:\\\\Data\\\\1-FAO ESTIMATION\\\\temperature\\2017-08-27.tif', '-te', '-54.6182781', '-26.7192200'

C:\\Data\\1-FAO ESTIMATION\\temperature\2017-09-30.tif
36
['gdalwarp', 'C:\\Users\\Administrator\\Desktop\\4-SKT_1km\\2017-09-30.tif', 'C:\\\\Data\\\\1-FAO ESTIMATION\\\\temperature\\2017-09-30.tif', '-te', '-54.6182781', '-26.7192200', '-48.0224681', '-22.5165800', '-r', 'near', '-ts', '2938', '1872', '-of', 'GTiff', '-ot', 'Float32', '-s_srs', 'EPSG:4326', '-t_srs', 'EPSG:4326']
C:\\Data\\1-FAO ESTIMATION\\temperature\2017-10-01.tif
37
['gdalwarp', 'C:\\Users\\Administrator\\Desktop\\4-SKT_1km\\2017-10-01.tif', 'C:\\\\Data\\\\1-FAO ESTIMATION\\\\temperature\\2017-10-01.tif', '-te', '-54.6182781', '-26.7192200', '-48.0224681', '-22.5165800', '-r', 'near', '-ts', '2938', '1872', '-of', 'GTiff', '-ot', 'Float32', '-s_srs', 'EPSG:4326', '-t_srs', 'EPSG:4326']
C:\\Data\\1-FAO ESTIMATION\\temperature\2017-10-02.tif
38
['gdalwarp', 'C:\\Users\\Administrator\\Desktop\\4-SKT_1km\\2017-10-02.tif', 'C:\\\\Data\\\\1-FAO ESTIMATION\\\\temperature\\2017-10-02.tif', '-te', '-54.6182781', '-26.71922

C:\\Data\\1-FAO ESTIMATION\\temperature\2017-11-05.tif
72
['gdalwarp', 'C:\\Users\\Administrator\\Desktop\\4-SKT_1km\\2017-11-05.tif', 'C:\\\\Data\\\\1-FAO ESTIMATION\\\\temperature\\2017-11-05.tif', '-te', '-54.6182781', '-26.7192200', '-48.0224681', '-22.5165800', '-r', 'near', '-ts', '2938', '1872', '-of', 'GTiff', '-ot', 'Float32', '-s_srs', 'EPSG:4326', '-t_srs', 'EPSG:4326']
C:\\Data\\1-FAO ESTIMATION\\temperature\2017-11-06.tif
73
['gdalwarp', 'C:\\Users\\Administrator\\Desktop\\4-SKT_1km\\2017-11-06.tif', 'C:\\\\Data\\\\1-FAO ESTIMATION\\\\temperature\\2017-11-06.tif', '-te', '-54.6182781', '-26.7192200', '-48.0224681', '-22.5165800', '-r', 'near', '-ts', '2938', '1872', '-of', 'GTiff', '-ot', 'Float32', '-s_srs', 'EPSG:4326', '-t_srs', 'EPSG:4326']
C:\\Data\\1-FAO ESTIMATION\\temperature\2017-11-07.tif
74
['gdalwarp', 'C:\\Users\\Administrator\\Desktop\\4-SKT_1km\\2017-11-07.tif', 'C:\\\\Data\\\\1-FAO ESTIMATION\\\\temperature\\2017-11-07.tif', '-te', '-54.6182781', '-26.71922

C:\\Data\\1-FAO ESTIMATION\\temperature\2017-12-11.tif
108
['gdalwarp', 'C:\\Users\\Administrator\\Desktop\\4-SKT_1km\\2017-12-11.tif', 'C:\\\\Data\\\\1-FAO ESTIMATION\\\\temperature\\2017-12-11.tif', '-te', '-54.6182781', '-26.7192200', '-48.0224681', '-22.5165800', '-r', 'near', '-ts', '2938', '1872', '-of', 'GTiff', '-ot', 'Float32', '-s_srs', 'EPSG:4326', '-t_srs', 'EPSG:4326']
C:\\Data\\1-FAO ESTIMATION\\temperature\2017-12-12.tif
109
['gdalwarp', 'C:\\Users\\Administrator\\Desktop\\4-SKT_1km\\2017-12-12.tif', 'C:\\\\Data\\\\1-FAO ESTIMATION\\\\temperature\\2017-12-12.tif', '-te', '-54.6182781', '-26.7192200', '-48.0224681', '-22.5165800', '-r', 'near', '-ts', '2938', '1872', '-of', 'GTiff', '-ot', 'Float32', '-s_srs', 'EPSG:4326', '-t_srs', 'EPSG:4326']
C:\\Data\\1-FAO ESTIMATION\\temperature\2017-12-13.tif
110
['gdalwarp', 'C:\\Users\\Administrator\\Desktop\\4-SKT_1km\\2017-12-13.tif', 'C:\\\\Data\\\\1-FAO ESTIMATION\\\\temperature\\2017-12-13.tif', '-te', '-54.6182781', '-26.71

IndexError: list index out of range